In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

In [2]:
#데이터 다운로드
df = pd.read_excel(r'C:\Users\Hon Kim\Desktop\관광통계\2019년_국민여행조사_원자료\dfr.xlsx', index = True) #불필요한 열 삭제
df.drop(['ID'],axis=1,inplace=True)
df.drop(['select_reason_2'],axis=1,inplace=True)
df.drop(['select_reason_3'],axis=1,inplace=True)

#일부 데이터 정수형변환
df['JEJU']=df['JEJU'].astype(int)
df['KYUNG_S']=df['KYUNG_S'].astype(int)
df['KYUNG_N']=df['KYUNG_N'].astype(int)
df['JEON_S']=df['JEON_S'].astype(int)
df['JEON_N']=df['JEON_N'].astype(int)
df['CHOONG_S']=df['CHOONG_S'].astype(int)
df['CHOONG_N']=df['CHOONG_N'].astype(int)
df['GANGWON']=df['GANGWON'].astype(int)
df['KYUNGGI']=df['KYUNGGI'].astype(int)
df['SEJONG']=df['SEJONG'].astype(int)
df['ULSAN']=df['ULSAN'].astype(int)
df['DAEJEON']=df['DAEJEON'].astype(int)
df['GWANGJU']=df['GWANGJU'].astype(int)
df['INCHEON']=df['INCHEON'].astype(int)
df['DAEGU']=df['DAEGU'].astype(int)
df['BUSAN']=df['BUSAN'].astype(int)
df['SEOUL']=df['SEOUL'].astype(int)


#데이터 구성요소확인
df['pre_food'].unique()

#결측치 처리
df=df.fillna(0)

#결측치 처리 확인(multi-hot encoding)
df['pre_food'].unique()


array([0., 2.])

In [3]:
#만족도, 재방문률, 추천정도 변수를 합쳐
#타겟변수설정
point = df['Satify']+df['revisit']+df['recommend']

#point2 = point >= 15
#(point2.values==True).sum()

#point2 = point >= 14
#(point2.values==True).sum()

#point2 = point >= 13
#(point2.values==True).sum()

point2 = point >= 12
(point2.values==True).sum()

#point2 = point >= 11
#(point2.values==True).sum()

#point2 = point >= 10
#(point2.values==True).sum()

7852

In [4]:
#데이터 프레임 상 타겟변수 제거
df.drop(['Satify'],axis=1,inplace=True)
df.drop(['revisit'],axis=1,inplace=True)
df.drop(['recommend'],axis=1,inplace=True)
df.head()

,visit_count,visit_acq,stay,COST,NUM,per_COST,cost_ski_watering,cost_camping,cost_hiking,cost_fishing,...,earn_self_classify_7,BMAR_1,BMAR_2,BMAR_3,BMAR_4,BMAR_5,winter,spring,summer,autumn
0,4,1,1,100000.0,1,100000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,0
1,3,1,1,250000.0,1,250000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
2,3,1,0,50001.0,3,16667.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,0,0
3,3,1,1,120000.0,1,120000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1
4,2,1,2,350001.0,3,116667.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,0


In [5]:
def mn(name):
    return df[name]/df[name].max()

In [6]:
for name in df.columns:
    df[name] = mn(name)

In [7]:
#훈련용 데이터셋, 시험 데이터셋 설정
X_train, X_test, y_train, y_test = train_test_split(df, point2, random_state=42, stratify = point2)
#훈련용 데이터셋 타겟변수 분포 확인
sum(y_train)/len(y_train)

0.7476196521518345

In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [46]:
# 각 레이어의 가중치와 바이어스를 초기화하는 옵션이 있음
# keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
# keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=333, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [47]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [48]:
clf.fit(X_train, y_train, batch_size=15, epochs=40)

Epoch 1/40
526/526 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 0.7457
Epoch 2/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3735 - accuracy: 0.8678
Epoch 3/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3435 - accuracy: 0.8742
Epoch 4/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3316 - accuracy: 0.8755
Epoch 5/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3275 - accuracy: 0.8770
Epoch 6/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3173 - accuracy: 0.8772
Epoch 7/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3110 - accuracy: 0.8788
Epoch 8/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3048 - accuracy: 0.8810
Epoch 9/40
526/526 [==============================] - 1s 1ms/step - loss: 0.3009 - accuracy: 0.8827
Epoch 10/40
526/526 [==============================] - 1s 1ms/step - loss: 0.2891 - accuracy: 0.8897

In [49]:
# epoch 10, train-accuracy: 0.921
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.6997844576835632, 0.8221629858016968]


In [65]:
# 각 레이어의 가중치와 바이어스를 초기화하는 옵션이 있음
# keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
# keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=333, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),

    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [66]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [67]:
clf.fit(X_train, y_train, batch_size=15, epochs=10)

Epoch 1/10
526/526 [==============================] - 1s 1ms/step - loss: 0.5016 - accuracy: 0.7584
Epoch 2/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3610 - accuracy: 0.8710
Epoch 3/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3249 - accuracy: 0.8799: 0s - loss: 0.3304 
Epoch 4/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3105 - accuracy: 0.8847
Epoch 5/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3008 - accuracy: 0.8875
Epoch 6/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2904 - accuracy: 0.8926
Epoch 7/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2778 - accuracy: 0.8992
Epoch 8/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2683 - accuracy: 0.9030
Epoch 9/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2542 - accuracy: 0.9078
Epoch 10/10
526/526 [==============================] - 1s 1ms/step - loss: 0.247

In [68]:
# epoch 10, train-accuracy: 0.921
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.44461843371391296, 0.8571972846984863]


In [69]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=333, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),

    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [5,10,15, 20,25,30]
epochs = [5,10,15,20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.864542 using {'batch_size': 5, 'epochs': 5}
0.864542 (0.008707) with: {'batch_size': 5, 'epochs': 5}
0.842198 (0.002967) with: {'batch_size': 5, 'epochs': 10}
0.831154 (0.002992) with: {'batch_size': 5, 'epochs': 15}
0.832420 (0.017898) with: {'batch_size': 5, 'epochs': 20}
0.858703 (0.005942) with: {'batch_size': 10, 'epochs': 5}
0.843596 (0.006554) with: {'batch_size': 10, 'epochs': 10}
0.832803 (0.012950) with: {'batch_size': 10, 'epochs': 15}
0.825569 (0.002398) with: {'batch_size': 10, 'epochs': 20}
0.860987 (0.003244) with: {'batch_size': 15, 'epochs': 5}
0.826076 (0.014103) with: {'batch_size': 15, 'epochs': 10}
0.836740 (0.003442) with: {'batch_size': 15, 'epochs': 15}
0.817316 (0.003875) with: {'batch_size': 15, 'epochs': 20}
0.858702 (0.006864) with: {'batch_size': 20, 'epochs': 5}
0.841309 (0.011853) with: {'batch_size': 20, 'epochs': 10}
0.830775 (0.009950) with: {'batch_size': 20, 'epochs': 15}
0.821631 (0.010035) with: {'batch_size': 20, 'epochs': 20}
0.859846 (0.